# Manifold Sculpting

In [1]:
import matplotlib.pyplot as plt
import torch

In [5]:
k = 2
data = torch.Tensor([[1,1],[7,7],[9,9],[12,12], [5,5], [2,2]])
p = data[2]

data_size = torch.tensor(data.size())
m = data_size[0] # number of data points
p_size = data_size[1:] # size of the single datapoint
p_size0 = p_size - torch.ones_like(p_size) # to be used for indexing
print(m)
print(p_size)

tensor(6)
tensor([2])


### Compute distances

In [6]:
x2 = data * data
x2 = x2.sum(axis=1)
# print(x2)
data_t = torch.transpose(data, 0, 1) # pb in dim >i (es foto...), credo funzioni cmq
xx = data@data_t
# print(xx)
# print(x2.unsqueeze(dim=1))
dist = torch.sqrt( x2 - 2*xx + x2.unsqueeze(dim=1) )
print(dist)

tensor([[ 0.0000,  8.4853, 11.3137, 15.5563,  5.6569,  1.4142],
        [ 8.4853,  0.0000,  2.8284,  7.0711,  2.8284,  7.0711],
        [11.3137,  2.8284,  0.0000,  4.2426,  5.6569,  9.8995],
        [15.5563,  7.0711,  4.2426,  0.0000,  9.8995, 14.1421],
        [ 5.6569,  2.8284,  5.6569,  9.8995,  0.0000,  4.2426],
        [ 1.4142,  7.0711,  9.8995, 14.1421,  4.2426,  0.0000]])


In [39]:
# extract the neighbours
_, indices = torch.sort(dist)
# print(indices)
# print(dist)
# keep the first k (without the point itself)
kneighb = indices[:, 1:k+1]
kdist   = torch.zeros_like(kneighb)

a = []
for i in range(m):
    # per qlche motivo nn si riesce a farlo direttamente con i torch.tensor
    a.append( dist[kneighb[i][:], i] )
# print(a)
kdist = torch.reshape( torch.cat( a, 0 ), (-1, k) )
del a

# print(kdist)

In [ ]:
# Compute angles
# Loop over points
# Variables with _idx run from 0 to m-1, they select points from `data`
# Variables without _idx run from 0 to k-1

# Entry i,j hosts the info (angle theta, kindex with respect to the neighborhood of j)
# about the most colinear point of the couple i-j ie i-j-l
theta = torch.ones((m,k))
colinear = torch.ones((m,k))
# Loop over data points
for i_idx in range(m):
    # Loop over neighbor points of i
    for j_kidx, j_idx in enumerate(kneighb[i_idx]): # che me ne faccio dl'enumerate
        p2nj = data[i_idx] - data[j_idx]
        p2nj /= torch.norm(p2nj)

        # angle = torch.ones(k)
        colinear_kidx =  torch.ones(k)
        # Loop over neighbor points of j
        for l_kidx, l_indx in enumerate(kneighb[j_idx]):
            p2nl = data[l_idx] - data[j_idx]
            p2nl /= torch.norm(p2nl)

            cos, cos_kidx = torch.sort(p2nl@c2nj, descending=True)
            # extract the collinear angle and neighbor, 0 is the maximum
            colinear[i, j_kidx] = cos_kidx[0] 
            theta[i, j_kidx] = torch.acos(cos[0]).item()
            # colinear_kidx[l_kidx] = cos_kidx[0] 
            # angle[l_kidx] = torch.acos(cos[0]).item()

dist_ave = torch.mean(kdist)

In [50]:
a = torch.Tensor([1,2])
b = torch.Tensor([2,3])
c = a@b
d = c.item()
a.tolist()
a[0]

tensor(1.)

In [65]:
from math import acos
a = torch.Tensor([0,1])
b = torch.Tensor([1,1])
a /= torch.norm(a)
b /= torch.norm(b)
print(b)
c = torch.acos(a@b)
c

tensor([0.7071, 0.7071])


tensor(0.7854)

In [ ]:
# PCA
cov = torch.cov(data)
eigenval, eigenvec = torch.linalg.eigh(-cov) # li ordina già in vista dla pca!
eigenval = -eigenval  # pké li ordina decrescenti
pca_basis = eigenvec@data

In [75]:
a = torch.Tensor([[1,1,0],
                  [1,4,0],
                  [0,0,3]])
eigenval, eigenvec = torch.linalg.eigh(-a)
eigenval = -eigenval
print(eigenval)
print(eigenvec)

tensor([4.3028, 3.0000, 0.6972])
tensor([[ 0.2898,  0.0000, -0.9571],
        [ 0.9571,  0.0000,  0.2898],
        [ 0.0000,  1.0000,  0.0000]])
